In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from tqdm import tqdm
from datetime import datetime
import json
from sklearn.preprocessing import LabelEncoder
import calendar
import datetime
warnings.filterwarnings("ignore")

# Web scrapping data from Spotify Chart

## Get date needed for each week

format of links to download:
https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download
https://spotifycharts.com/regional/global/weekly/2016-12-30--2017-01-06/download

We see that all the dates that appear in the link are Fridays, each being one week apart

In [23]:
#An example of Calendar package
x = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2025, month)
    week1 = mycal[1]
    week2 = mycal[2]
      
    if week1[calendar.MONDAY] != 0:
        auditday = week1[calendar.MONDAY]
    else: 
        auditday = week2[calendar.MONDAY] 
    print("%10s %2d" % (calendar.month_name[month], auditday))
    x.append("%10s %2d" % (calendar.month_name[month], auditday))

   January  6
  February  3
     March  3
     April  7
       May  5
      June  2
      July  7
    August  4
 September  8
   October  6
  November  3
  December  8


In [4]:
# reference
c = calendar.TextCalendar(calendar.MONDAY)
str1 = c.formatmonth(2017,1)
print(str1)

    January 2017
Mo Tu We Th Fr Sa Su
                   1
 2  3  4  5  6  7  8
 9 10 11 12 13 14 15
16 17 18 19 20 21 22
23 24 25 26 27 28 29
30 31



In [5]:
(calendar.monthcalendar(2017,1)[0][calendar.FRIDAY],calendar.monthcalendar(2017,1)[4][calendar.FRIDAY])
#calendar.monthcalendar(2017,1)[c(0,4)][calendar.FRIDAY]

(0, 27)

In [6]:
#All the fridays in 2017
Fridays17 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2017, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2017, month, mycal[week][calendar.FRIDAY])
            Fridays17.append((2017, month, mycal[week][calendar.FRIDAY]))
            
            
Fridays17[0]

(2017, 1, 6)

In [7]:
#All the fridays in 2018
Fridays18 = []

for month in range(1, 13):
    mycal = calendar.monthcalendar(2018, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2018, month, mycal[week][calendar.FRIDAY])
            Fridays18.append((2018, month, mycal[week][calendar.FRIDAY]))


Fridays18[0][2]

5

In [8]:
#fridays in 2019 (Jan to Aug)
Fridays19 = []

for month in range(1, 9):
    mycal = calendar.monthcalendar(2019, month)
    
    for week in range(0,len(mycal)):
        if mycal[week][calendar.FRIDAY] != 0:
            #print(2019, month, mycal[week][calendar.FRIDAY])
            Fridays19.append((2019, month, mycal[week][calendar.FRIDAY]))

Fridays19[2]

(2019, 1, 18)

In [19]:
#Fridays in 2016
Fridays16to19 = [(2016, 12, 23),(2016, 12, 30)]

#Compile all the fridays 2016-2019
Fridays16to19 = Fridays16to19 + Fridays17 + Fridays18 + Fridays19
Fridays16to19[0]

(2016, 12, 23)

In [21]:
#formatting friday dates: 2016-12-23--2016-12-30

Fridays_format = []

for i in range(0, len(Fridays16to19)):
    year = Fridays16to19[i][0]
    month = Fridays16to19[i][1]
    day = Fridays16to19[i][2]  
    
    if month < 10:
        month = str(0)+str(month)
    else:
        month = str(month)
        
    if day < 10:
        day = str(0)+str(day)
    else:
        day = str(day)
    
    #print(str(year)+"-"+month+"-"+day)
    Fridays_format.append(str(year)+"-"+month+"-"+day)
    
Fridays_format[0]

'2016-12-23'

In [22]:
#put all the dates into the format of download link: 
#https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download

####################IndexError: list index out of range
# for j in range(0, len(Fridays_format)):
#     str_p1 = "https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] + "--"
#     if j != len(Fridays_format):
#         print(str_p1 + Fridays_format[j+1] + "/download")
        
# for j in range(0, len(Fridays_format)):
#     print("https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] 
#           + "--" + Fridays_format[j+1] + "/download")


j = 0
while(j<len(Fridays_format)-1):
    str_p1 = print("https://spotifycharts.com/regional/global/weekly/" + Fridays_format[j] + 
                   "--" + Fridays_format[j+1] + "/download")
    j += 1

https://spotifycharts.com/regional/global/weekly/2016-12-23--2016-12-30/download
https://spotifycharts.com/regional/global/weekly/2016-12-30--2017-01-06/download
https://spotifycharts.com/regional/global/weekly/2017-01-06--2017-01-13/download
https://spotifycharts.com/regional/global/weekly/2017-01-13--2017-01-20/download
https://spotifycharts.com/regional/global/weekly/2017-01-20--2017-01-27/download
https://spotifycharts.com/regional/global/weekly/2017-01-27--2017-02-03/download
https://spotifycharts.com/regional/global/weekly/2017-02-03--2017-02-10/download
https://spotifycharts.com/regional/global/weekly/2017-02-10--2017-02-17/download
https://spotifycharts.com/regional/global/weekly/2017-02-17--2017-02-24/download
https://spotifycharts.com/regional/global/weekly/2017-02-24--2017-03-03/download
https://spotifycharts.com/regional/global/weekly/2017-03-03--2017-03-10/download
https://spotifycharts.com/regional/global/weekly/2017-03-10--2017-03-17/download
https://spotifycharts.com/re